In [ ]:
import tweepy
import nltk
from nltk import sentiment
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def score_compound(tweet):
    vs = analyzer.polarity_scores(tweet)
    return (vs['compound'])

In [ ]:
scoredTweets_compound_only = [(bson[0], (score_compound(bson[0])), bson[1]) for bson in alltweets]

In [ ]:
import sys
import json
import pymongo
from datetime import datetime
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from pymongo import MongoClient

## insert your mongoDB access key
client = MongoClient('KEY HERE')

db = client.DNC
DNC = db.Test
class StdOutListener(StreamListener):
    def on_data(self, data):
        full_tweet = json.loads(data)
        ##this makes sure that you won't get clipped tweets
        if 'extended_tweet' in full_tweet:
            tweet_text = full_tweet.get('full_text')
        else:
            tweet_text = full_tweet.get('text')
        if 'id' in full_tweet:
            tweet_user = full_tweet.get('user')
            tweet_user_id = tweet_user['id']
        else:
            tweet_user_id = 'unknown'
        tweet_time = full_tweet.get('created_at')
        tweet_lang = full_tweet.get('lang')
        
        if tweet_lang != None and tweet_lang == 'en' and tweet_text != None and 'RT @' not in tweet_text:
            ##this is only taking the text and the time stamp, which is making the DB very space effecient
            score = score_compound(tweet_text)
            tweetObject = {
            "text": tweet_text,
            "time": tweet_time,
            "user_id": tweet_user_id,
            "score": score
            }
            DNC.insert_one(full_tweet)
            return True
    def on_error(self, status):
        print (status)
        
keyFile = open('YOUR TWITTER DEV KEY HERE','r')
CONSUMER_KEY = keyFile.readline().rstrip()
CONSUMER_SECRET = keyFile.readline().rstrip()
ACCESS_TOKEN_KEY = keyFile.readline().rstrip()
ACCESS_TOKEN_SECRET = keyFile.readline().rstrip()

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

while True:
    try:
        stream = Stream(auth, StdOutListener())
        stream.filter(track=["DNC", "Biden", "Harris", "Kamala","Trump","Democrats","Republicans","Buttigieg","Cory Booker","Tammy Duckworth","Bloomberg"]) ##insert your filters here
    except KeyboardInterrupt:
        s.close()
        stream.disconnect()
        break